# Homework01

### Problem 1
Using introspection as a tool to try and develop AI to model human cognitive processes is certainly an interesting idea, but I think it would fall short of the goal it tries to achieve. When we analyze our own thoughts and mental states, even if we capture a bit of what that state means it has no chance of figuring out everything about that state. Our own understanding of it will be nowhere near enough to create a comparison using a computer. Our brains are simply too complex for us to understand, and if we cannot understand them how are we to program computers to model them.
However, I think the best application for introspection in AI would be chatbots. If we can use introspection while we are having conversation and figure out what triggers state changes in our brain: what are the differences in mental state when we make smalltalk with someone on the path versus participating in a group discussion in class? Understanding these kinds of differences in mental state would help a chatbot choose better words for certain situations.

### Problem 2
My states are lists of the form [0, 1, 2, 3, ... ] where each element is a city. The way to transition from state to state is by swapping two cities in the ordering of the list. I keep track of the distances between the cities in a pandas Dataframe object.

In [ ]:
from search import Problem, hill_climbing, simulated_annealing, exp_schedule
import random
import pandas as pd
import numpy as np
from copy import deepcopy

class TSP(Problem):

    """
    The Traveling Salesman Problem
    ------------------------------
    To initialize the TSP object you need a dataframe using the Pandas library
    cities = pd.DataFrame({
        0: {1: 10, 2: 1, 3: 4},
        1: {0: 10, 2: 5, 3: 1},
        2: {0: 1, 1: 5, 3: 7},
        3: {0: 4, 1: 1, 2: 7}
    })
    So the distance between city 0 and city 1 is 10
    State: Current ordering of the cities
    state = [0, 1, 2, 3]
    """

    def __init__(self, cities):
        self.cities = cities
        self.initial = list(range(len(cities)))
        self.n_cities = len(cities)

    def value(self, state):
        val = 0
        for i in range(len(self.cities)-1):
            # Add next city distance to result
            val += self.cities[state[i]][state[i+1]]
        # Last to first
        val += self.cities[state[len(self.cities)-1]][state[0]]
        return -val

    def actions(self, state):
        action_list = []
        for i in range(10):
            first = random.randint(1, self.n_cities - 1)
            # Make sure we are not swapping the same city
            while True:
                second = random.randint(1, self.n_cities - 1)
                if first != second:
                    break
            action_list.append([first, second])
        return action_list

    def result(self, state, action):
        new_state = state[:]
        new_state[action[0]], new_state[action[1]] = new_state[action[1]], new_state[action[0]]
        return new_state


In [ ]:
def build_cities(n = 10):
    """
    build_cities creates a symmetric pandas dataframe with distances between 1 and 100.
    """
    cities = pd.DataFrame(np.zeros(shape=(n, n), dtype=np.int8))
    for i in range(n):
        for j in range(i, n):
            distance = random.randint(1, 100)
            cities[i][j] = distance
            cities[j][i] = distance
    return cities


if __name__ == "__main__":
    cities = build_cities(100)

    tsp = TSP(cities)

    hill_solution = hill_climbing(tsp)
    print('Hill-climbing solution       x: ' + str(hill_solution)
          + '\tvalue: ' + str(tsp.value(hill_solution))
          )

    annealing_solution = simulated_annealing(
        tsp,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )
    print('Simulated annealing solution x: ' + str(annealing_solution)
          + '\tvalue: ' + str(tsp.value(annealing_solution))
          )

Hill-climbing solution       x: [0, 81, 2, 3, 4, 5, 6, 7, 8, 60, 10, 11, 12, 13, 14, 55, 16, 17, 18, 57, 20, 21, 77, 23, 24, 25, 64, 27, 28, 1, 30, 31, 32, 33, 87, 75, 22, 37, 38, 39, 68, 41, 42, 43, 44, 45, 46, 47, 48, 49, 59, 51, 52, 53, 54, 85, 56, 19, 84, 50, 9, 61, 74, 63, 26, 65, 66, 80, 40, 69, 92, 71, 72, 73, 70, 35, 76, 36, 78, 79, 67, 29, 82, 58, 83, 15, 86, 34, 98, 89, 90, 91, 96, 93, 94, 95, 62, 97, 88, 99]	value: -3363
Simulated annealing solution x: [0, 1, 2, 28, 4, 5, 84, 7, 88, 13, 10, 29, 23, 76, 14, 58, 56, 27, 67, 54, 22, 15, 57, 83, 20, 31, 30, 61, 3, 11, 79, 69, 36, 55, 80, 51, 85, 39, 77, 68, 98, 71, 18, 47, 21, 90, 46, 12, 17, 48, 38, 41, 26, 24, 50, 63, 74, 42, 64, 82, 60, 92, 37, 89, 43, 45, 66, 99, 78, 35, 8, 44, 33, 91, 70, 72, 19, 75, 9, 16, 73, 53, 81, 97, 87, 32, 25, 62, 49, 59, 34, 86, 96, 94, 93, 6, 52, 95, 40, 65]	value: -1676


We can see that the Simulated annealing solution works much better than the hill climbing solution. This is because there might be no randomly chosen actions for the hill climbing to improve the current situation, whereas the annealing solution has the ability to choose a worse path for the chance to get to an even higher value later.

### Problem 3


In [4]:
from csp import CSP, parse_neighbors, min_conflicts, backtracking_search, mrv, forward_checking

def Scheduler():

    Courses = 'cs108 cs112 cs212 cs214 cs262 cs232 cs384'.split()
    Professors = 'Adams VanderLinden Norman Schuurman Plantinga'.split()
    TimeSlots = 'mwf800 mwf900 mwf1030 mwf1130'.split()
    Classrooms = 'sb382 nh253'.split()

    values = []
    for time in TimeSlots:
        for classroom in Classrooms:
            for prof in Professors:
                values.append([time, classroom, prof])

    # Set up domains for each class
    domains = {}
    for var in Courses:
        domains[var] = values

    # Set up each class as a neighbor for the other classes
    neighbors = {}
    for type in [Courses]:
        for A in type:
            neighbors[A] = []
    for type in [Courses]:
        for A in type:
            for B in type:
                if A != B:
                    if B not in neighbors[A]:
                        neighbors[A].append(B)
                    if A not in neighbors[B]:
                        neighbors[B].append(A)

    def scheduler_constraint(A, a, B, b, recurse=0):
        prof_to_course = {
            'cs108': "Schuurman",
            'cs112': "Adams",
            'cs212': "Plantinga",
            'cs214': "Adams",
            'cs232': "Norman",
            'cs262': "VanderLinden",
            'cs384': "Schuurman"
        }
        if prof_to_course[A] != a[2] or prof_to_course[B] != b[2]:  # Prof isn't teaching the right course
            return False
        if a[2] == b[2] and a[0] == b[0]:  # Professor is the same and TimeSlot is the same
            return False
        if a[1] == b[1] and a[0] == b[0]:  # Classroom is the same and TimeSlot is the same
            return False
        return True
    return CSP(Courses, domains, neighbors, scheduler_constraint)

For this problem I used the various classes as the variables and set the domain to a triple containing Professor, TimeSlot, and Classroom.

In [5]:
def print_solution(result):
    """A CSP solution printer copied from csp.py."""
    Courses = 'cs108 cs112 cs212 cs214 cs262 cs232 cs384'.split()
    for course in list(Courses):
        print('Course:', course)
        for (var, val) in result.items():
            if var == course:
                for i in val: print('\t', i)


if __name__ == "__main__":

    puzzle = Scheduler()
    result = backtracking_search(puzzle, select_unassigned_variable=mrv, inference=forward_checking)

    if puzzle.goal_test(puzzle.infer_assignment()):
        print("Solution:\n")
        print_solution(result)
    else:
        print("failed...")
        print(puzzle.curr_domains)
        puzzle.display(puzzle.infer_assignment())

Solution:

Course: cs108
	 mwf900
	 sb382
	 Schuurman
Course: cs112
	 mwf1130
	 sb382
	 Adams
Course: cs212
	 mwf900
	 nh253
	 Plantinga
Course: cs214
	 mwf1030
	 sb382
	 Adams
Course: cs262
	 mwf800
	 nh253
	 VanderLinden
Course: cs232
	 mwf1030
	 nh253
	 Norman
Course: cs384
	 mwf800
	 sb382
	 Schuurman


We can see that every course gets its corresponding Professor and a unique TimeSlot/Classroom combination.